In [7]:
import pandas as pd
import os
import joblib
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing import sequence
import pandas as pd
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
from itertools import groupby
import regex as re
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO 
from pyvi import ViTokenizer, ViPosTagger,ViUtils
import joblib
from scipy.sparse import hstack
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing
import os

In [30]:
def annotations(dataset):
  pos = []
  max_len = 8000
  for i in range(dataset.shape[0]):
    n = len(dataset.at[i,'cmt'])
    l = [0] * max_len
    for j in range(dataset.at[i,'start_index'], dataset.at[i,'end_index']):
      l[j] = 1
    pos.append(l)
  return pos


def abbreviation_predict(t):
  model_path = os.path.abspath('../model/abb_model.sav')
  loaded_model = joblib.load(model_path)

  da_path = os.path.abspath('../dictionary/abbreviation_dictionary_vn.xlsx')
  train_path = os.path.abspath('../dictionary/train_duplicate_abb_data.xlsx')
  dev_path = os.path.abspath('../dictionary/dev_duplicate_abb_data.xlsx')
  test_path = os.path.abspath('../dictionary/test_duplicate_abb_data.xlsx')
  duplicate_abb = pd.read_excel(da_path,sheet_name='duplicate',header=None)
  duplicate_abb = list(duplicate_abb[0])
    
  train_duplicate_abb_data = pd.read_excel(train_path)
  dev_duplicate_abb_data = pd.read_excel(dev_path)
  test_duplicate_abb_data = pd.read_excel(test_path)
  duplicate_abb_data = pd.concat([train_duplicate_abb_data, dev_duplicate_abb_data, test_duplicate_abb_data], ignore_index=True)
  duplicate_abb_data = duplicate_abb_data.drop_duplicates(keep='last').reset_index(drop=True)

  X = duplicate_abb_data[['abb','start_index','end_index','cmt']]
  y = duplicate_abb_data['origin']

  from sklearn import preprocessing
  le = preprocessing.LabelEncoder()
  y = le.fit_transform(y)
  enc = DictVectorizer()
  Tfidf_vect = TfidfVectorizer(max_features=1200)

  temp = annotations(X)
  X_pos = sparse.csr_matrix(np.asarray(temp))
  X_abb = enc.fit_transform(X[['abb']].to_dict('records'))
  X_text= Tfidf_vect.fit_transform(X['cmt'])
  X = hstack((X_abb,X_pos,X_text))

  text = str(t)
  max_len = 8000
  if len(t)>max_len:
    text =t[:max_len]
    
  cmt = ' ' + text + ' '
  for abb in duplicate_abb:   
    start_index = 0
    count = 0
    while start_index >-1: #start_index = -1 -> abb is not in cmt
      start_index = cmt.find(' '+abb+' ')    #find will return FIRST index abb in cmt  
      if start_index > -1:      
        end_index = start_index + len(abb)
        t = pd.DataFrame([[abb, start_index, end_index, text]], columns=['abb', 'start_index', 'end_index','cmt'],index=None)
        temp = annotations(t)
        X_pos = sparse.csr_matrix(np.asarray(temp))
        
        X_abb = enc.transform(t[['abb']].to_dict('records'))
        # print(t['cmt'])
        X_text= Tfidf_vect.transform([text])
        
        X = hstack((X_abb,X_pos,X_text))
        predict = loaded_model.predict(X)
        origin = le.inverse_transform(predict.argmax(axis=1))
        origin = ''.join(origin)
        text = text[:start_index+count*(len(origin)-len(abb))]+origin+text[end_index+count*(len(origin)-len(abb)):]
        text = ''.join(text)
        count = count + 1
        for i in range(start_index+1,end_index+1):#replace abb to space ' '
          cmt = cmt[:i] + ' '+ cmt[i+1:]        
  return text

In [31]:
abbreviation_predict('1:54 abe nha bắt cô panda dội nc đái :))))')

'1:54 abe nha bắt cô panda dội nước đái :))))'

In [6]:
tknz_path = os.path.abspath('../model/tokenizer.pickle')
with open(tknz_path,"rb") as f:
    tokenizer = pickle.load(f)

In [24]:
test = 'Chúng ta đã đi qua quãng thời gian đẹp nhất của nhau. Xa nhau là điều 2 đứa không hề muốn. Thôi thì coi như mình có duyên gặp nhau mà không đủ phận ở bên nhau. Em nợ anh 1 câu yêu thương cho mai này, xin hẹn nhau một kiếp sống khác ta sum vầy... Bài hát đúng tâm trạng. Cảm ơn nhé. ☘️ ☘️ ☘️. Chúng ta đã đi qua quãng thời gian đẹp nhất của nhau. Xa nhau là điều 2 đứa không hề muốn. Thôi thì coi như mình có duyên gặp nhau mà không đủ phận ở bên nhau. Em nợ anh 1 câu yêu thương cho mai này, xin hẹn nhau một kiếp sống khác ta sum vầy... Bài hát đúng tâm trạng. Cảm ơn nhé. ☘️ ☘️ ☘️'
test = preprocessing(test)
test = tokenizer.texts_to_sequences([test])
test = sequence.pad_sequences(test,maxlen=80)

In [27]:
tokenizer.sequences_to_texts(test)

['cho mai này xin hẹn nhau một kiếp sống khác ta sum_vầy bài hát đúng tâm_trạng cảm_ơn nhé ️ ️ ️ chúng_ta đã đi quá quãng thời_gian đẹp nhất của nhau xa nhau là điều đứa không hề muốn thôi_thì coi như mình có duyên gặp nhau mà không đủ phận ở bên nhau em nợ anh câu yêu_thương cho mai này xin hẹn nhau một kiếp sống khác ta sum_vầy bài hát đúng tâm_trạng cảm_ơn nhé ️ ️ ️']

In [8]:
model_path = os.path.abspath('../model/Text_CNN_model_PhoW2V.h5')
model = tf.keras.models.load_model(model_path)

In [14]:
model.predict(test)

array([[0.1911226 , 0.63787633, 0.17100114]], dtype=float32)

In [2]:
a = 'Huy là ta'
preprocessing(a)

C:\Users\Huy\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator Ridge from version 0.22.2.post1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


'huy là ta'

In [7]:
preprocessing('mn nghe rõ không :)))')

'mọi người nghe rõ không'

In [4]:
da_path = os.path.abspath('../dictionary/abbreviation_dictionary_vn.xlsx')
duplicate_abb = pd.read_excel(da_path,sheet_name='duplicate',header=None)

In [1]:
import sys

In [5]:
n = len(sys.argv)

In [7]:
# Arguments passed
print("\nName of Python script:", sys.argv[0])
 
print("\nArguments passed:", end = " ")
for i in range(1, n):
    print(sys.argv[i], end = " ")


Name of Python script: C:\Users\Huy\anaconda3\envs\Homura2\lib\site-packages\ipykernel_launcher.py

Arguments passed: -f C:\Users\Huy\AppData\Roaming\jupyter\runtime\kernel-e800c74a-48fd-4744-9655-32cf5ed63009.json 

In [1]:
def gen():
    yield 1

In [6]:
gen = (x*x for x in range(3))

In [9]:
for i in gen:
    print(i)

0
1
4


In [11]:
def gen():
    for i in range(3):
        yield i*i

gene = gen()
for i in gene:
    print(i)

0
1
4


In [12]:
for i in gene:
    print(i)